In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

from preprocessing import load_suffixes, prepare_input
from recalibrator import Recalibrator
from utils import match
from confidence_intervals import confidence_intervals

In [6]:
trios = ["ajt", "chd", "corpas", "yri"]

# Pre-processing. Uncomment during first run of the script, then
# comment to avoid re-computing

# for trio in trios:
#     data_dir = '../data/' + trio + '/'
#     df = load_suffixes(data_dir)
#     df.to_csv(trio + '.csv', index=False)

  0%|          | 0/12 [00:00<?, ?it/s]/home/boi/research/ML-maternal-cell-contamination/python/preprocessing.py:196: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  df = df.convert_objects(convert_numeric=True)
100%|██████████| 12/12 [01:42<00:00,  8.53s/it]


For each trio, construct a training set from the remaining trios and save the predictions into a dictionary.

In [7]:

results = {}

for train in trios:
    print("Train trio: {}".format(train))
    results[train] = {}
    
    df_train = pd.read_csv(train + '.csv')    
    X_train = prepare_input(df_train, target_cols=['justchild^GT'])
    y_train = df_train['justchild^GT'].values


    r = Recalibrator()
    r.train(X_train, y_train);
    
    for test in trios:
        if train == test:
            continue
            
        print("|--- Test trio: {}".format(train))
            
        results[train][test] = {}
        
        
        df_test = pd.read_csv(test + '.csv')
        X_test = prepare_input(df_test, target_cols=['justchild^GT'])
        y_test = df_test['justchild^GT'].values

        contaminations = df_test['contamination'].values
        contamination_values = list(sorted(np.unique(contaminations)))

        idx = {}

        for contamination in contamination_values:
            idx[contamination] = contaminations == contamination

        results[train][test]['y_test'] = y_test
        results[train][test]['preds_naive'] = df_test['abortus^GT'].values
        results[train][test]['idx'] = idx
        
        results[train][test]['preds_lr'] = r.predict_lr(X_test)
        results[train][test]['preds_xgb'] = r.predict_xgb(X_test)
        results[train][test]['preds_ci'] = confidence_intervals(df_test)



Train trio: ajt


/usr/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Training logistic regression


/usr/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Training XGB
[0]	validation_0-merror:0.020068
Will train until validation_0-merror hasn't improved in 20 rounds.
[1]	validation_0-merror:0.019884
[2]	validation_0-merror:0.018432
[3]	validation_0-merror:0.017115
[4]	validation_0-merror:0.016672
[5]	validation_0-merror:0.016365
[6]	validation_0-merror:0.016278
[7]	validation_0-merror:0.015897
[8]	validation_0-merror:0.015429
[9]	validation_0-merror:0.015306
[10]	validation_0-merror:0.015343
[11]	validation_0-merror:0.015196
[12]	validation_0-merror:0.015048
[13]	validation_0-merror:0.01479
[14]	validation_0-merror:0.014937
[15]	validation_0-merror:0.014617
[16]	validation_0-merror:0.01458
[17]	validation_0-merror:0.014273
[18]	validation_0-merror:0.014174
[19]	validation_0-merror:0.014002
[20]	validation_0-merror:0.013941
[21]	validation_0-merror:0.01367
[22]	validation_0-merror:0.013596
[23]	validation_0-merror:0.013399
[24]	validation_0-merror:0.013301
[25]	validation_0-merror:0.013276
[26]	validation_0-merror:0.013153
[27]	validation

...

/usr/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/boi/research/ML-maternal-cell-contamination/python/confidence_intervals.py:10: RuntimeWarning: divide by zero encountered in true_divide
  lower_bound = contaminations - z*np.sqrt(contaminations*(1 - contaminations)/df_test[ab_name + '^DP'].values)
/home/boi/research/ML-maternal-cell-contamination/python/confidence_intervals.py:11: RuntimeWarning: divide by zero encountered in true_divide
  upper_bound = contaminations + z*np.sqrt(contaminations*(1 - contaminations)/df_test[ab_name + '^DP'].values)


|--- Test trio: yri


/usr/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


|--- Test trio: yri


/usr/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [9]:
import pickle

with open("results_1v1.pickle", "wb") as f:
    pickle.dump(results, f)

In [16]:
df_train

,#CHROM,POS,REF,ALT,AC,AF,abortus^GT,abortus^DP,father^GT,father^DP,...,abortus^GT^1^1H,abortus^GT^2^1H,father^GT^0^1H,father^GT^1^1H,father^GT^2^1H,mother^GT^0^1H,mother^GT^1^1H,mother^GT^2^1H,contamination,justchild^GT
0,chr1,183238,3,2,1,0.25,0,9,0,14,...,0,0,1,0,0,0,1,0,0.50,0
10,chr1,966728,2,4,2,0.25,1,100,1,97,...,1,0,0,1,0,1,0,0,0.50,1
20,chr1,999842,2,1,4,0.75,1,50,2,60,...,1,0,0,0,1,0,1,0,0.50,2
30,chr1,1091327,2,1,4,0.75,1,53,2,50,...,1,0,0,0,1,0,1,0,0.50,2
40,chr1,1327586,2,4,6,1.00,2,125,2,148,...,0,1,0,0,1,0,0,1,0.50,2
50,chr1,1703565,4,2,3,0.50,1,156,0,94,...,1,0,1,0,0,0,0,1,0.50,1
60,chr1,1739519,2,3,5,0.75,2,15,2,19,...,0,1,0,0,1,0,1,0,0.50,2
70,chr1,1942086,4,2,3,0.50,1,57,2,70,...,1,0,0,0,1,1,0,0,0.50,1
80,chr1,2056554,4,2,3,0.50,1,109,1,105,...,1,0,0,1,0,0,1,0,0.50,1
90,chr1,2556714,1,3,3,0.50,1,48,1,79,...,1,0,0,1,0,0,1,0,0.50,1


In [10]:

train = 'ajt'

df_train = pd.read_csv(train + '.csv')    
X_train = prepare_input(df_train, target_cols=['justchild^GT'])
y_train = df_train['justchild^GT'].values


r = Recalibrator()
r.train(X_train, y_train);

test = 'yri'

df_test = pd.read_csv(test + '.csv')
X_test = prepare_input(df_test, target_cols=['justchild^GT'])
y_test = df_test['justchild^GT'].values

contaminations = df_test['contamination'].values
contamination_values = list(sorted(np.unique(contaminations)))

idx = {}

for contamination in contamination_values:
    idx[contamination] = contaminations == contamination

# results[train][test]['y_test'] = y_test
# results[train][test]['preds_naive'] = df_test['abortus^GT'].values
# results[train][test]['idx'] = idx

# results[train][test]['preds_lr'] = r.predict_lr(X_test)
# results[train][test]['preds_xgb'] = r.predict_xgb(X_test)
# results[train][test]['preds_ci'] = confidence_intervals(df_test)



/usr/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Training logistic regression


/usr/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Training XGB
[0]	validation_0-merror:0.020068
Will train until validation_0-merror hasn't improved in 20 rounds.
[1]	validation_0-merror:0.019884
[2]	validation_0-merror:0.018432
[3]	validation_0-merror:0.017115
[4]	validation_0-merror:0.016672
[5]	validation_0-merror:0.016365
[6]	validation_0-merror:0.016278
[7]	validation_0-merror:0.015897
[8]	validation_0-merror:0.015429
[9]	validation_0-merror:0.015306
[10]	validation_0-merror:0.015343
[11]	validation_0-merror:0.015196
[12]	validation_0-merror:0.015048
[13]	validation_0-merror:0.01479
[14]	validation_0-merror:0.014937
[15]	validation_0-merror:0.014617
[16]	validation_0-merror:0.01458
[17]	validation_0-merror:0.014273
[18]	validation_0-merror:0.014174
[19]	validation_0-merror:0.014002
[20]	validation_0-merror:0.013941
[21]	validation_0-merror:0.01367
[22]	validation_0-merror:0.013596
[23]	validation_0-merror:0.013399
[24]	validation_0-merror:0.013301
[25]	validation_0-merror:0.013276
[26]	validation_0-merror:0.013153
[27]	validation

[236]	validation_0-merror:0.007346
[237]	validation_0-merror:0.007333
[238]	validation_0-merror:0.007358
[239]	validation_0-merror:0.007309
[240]	validation_0-merror:0.007284
[241]	validation_0-merror:0.007296
[242]	validation_0-merror:0.007284
[243]	validation_0-merror:0.007272
[244]	validation_0-merror:0.007296
[245]	validation_0-merror:0.007309
[246]	validation_0-merror:0.007284
[247]	validation_0-merror:0.007284
[248]	validation_0-merror:0.007309
[249]	validation_0-merror:0.007309
[250]	validation_0-merror:0.007284
[251]	validation_0-merror:0.007309
[252]	validation_0-merror:0.007272
[253]	validation_0-merror:0.007272
[254]	validation_0-merror:0.007272
[255]	validation_0-merror:0.007284
[256]	validation_0-merror:0.007259
[257]	validation_0-merror:0.007223
[258]	validation_0-merror:0.007223
[259]	validation_0-merror:0.007272
[260]	validation_0-merror:0.007296
[261]	validation_0-merror:0.007309
[262]	validation_0-merror:0.007296
[263]	validation_0-merror:0.007309
[264]	validation_0-m

In [11]:
X_test

array([[1, 3, 4, ..., False, True, 0.03],
       [4, 2, 1, ..., True, False, 0.03],
       [2, 3, 2, ..., True, False, 0.03],
       ...,
       [3, 0, 2, ..., True, False, 0.2],
       [0, 3, 1, ..., False, False, 0.2],
       [3, 4, 6, ..., False, True, 0.2]], dtype=object)

In [13]:
preds_lr = r.predict_lr(X_test)

/usr/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [14]:
accuracy_score(y_test, preds_lr)

0.9729878808878056